In [2]:
import tensorflow as tf
import io

policy_dir = '../output/policy'
saved_policy = tf.compat.v2.saved_model.load(policy_dir)

In [3]:
saved_policy

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x1451d9128>

In [8]:
import IPython
import base64

def embed_gif(gif_buffer):
  """Embeds a gif file in the notebook."""
  tag = '<img src="data:image/gif;base64,{0}"/>'.format(base64.b64encode(gif_buffer).decode())
  return IPython.display.HTML(tag)

def run_episodes_and_create_video(policy, eval_tf_env, eval_py_env):
  num_episodes = 3
  frames = []
  for _ in range(num_episodes):
    time_step = eval_tf_env.reset()
    frames.append(eval_py_env.render())
    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = eval_tf_env.step(action_step.action)
      frames.append(eval_py_env.render())
  gif_file = io.BytesIO()
  imageio.mimsave(gif_file, frames, format='gif', fps=60)
  IPython.display.display(embed_gif(gif_file.getvalue()))

In [5]:
import gym
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
import imageio
import io


eval_py_env = suite_gym.load('CartPole-v1')
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)



In [7]:
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      dist_from_center = abs(time_step.observation[0])
      episode_return += time_step.reward - dist_from_center 
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]

compute_avg_return(eval_env, saved_policy)

356.61484

In [ ]:
run_episodes_and_create_video(saved_policy, eval_env, eval_py_env)

